<a href="https://colab.research.google.com/github/JoshuaTewolde/Weather-Forecasting/blob/main/Lagging_ML_Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv

import pandas as pd
import glob
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score


In [ ]:
# Load all CSVs
files = glob.glob("/content/drive/MyDrive/Weather Datasets/*.csv")
print("This many files: ", len(files))

df_list = []
for f in files:
    df_list.append(pd.read_csv(f, low_memory=False))

df = pd.concat(df_list, ignore_index=True)


This many files:  7


In [ ]:
#replace M with NaN
df = df.replace("M", np.nan)

# Drop rows where temperature is missing
df = df.dropna(subset=["tmpf"])


In [ ]:
#convert to numeric
numeric_cols = ["tmpf", "dwpf", "relh",
                "feel", "drct", "sped", "mslp",
                "lon", "lat"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")


In [ ]:
#convert date/time
df["valid"] = pd.to_datetime(df["valid"])
df = df.rename(columns={"valid": "datetime",})
df["DOY"] = df["datetime"].dt.dayofyear #day of the year
df["TOD"] = df["datetime"].dt.hour*60 + df["datetime"].dt.minute #time of day
# df = df.drop(columns=["datetime"])
# df = df.drop(columns=["valid"])

#sort by date
df = df.sort_values("datetime")


In [ ]:
target = df[df['station'] == 'FNT'].copy()
predictors = df.copy()

#target_time is new column with the time is 24 hrs in the future
predictors['target_time'] = predictors['datetime'] + pd.Timedelta(hours=24)

#rename the timestamp column in the target to "flint_time"
target = target.rename(columns={'datetime': 'flint_time', "tmpf": "target_tmpf",})


In [ ]:
print(predictors.shape)
print(target.shape)
print(predictors["target_time"].is_unique)
print(target["flint_time"].is_unique)


(18353896, 15)
(124725, 14)
False
False


In [ ]:
df_merged = pd.merge_asof(
    predictors.sort_values('target_time'),
    target.sort_values('flint_time'),
    left_on='target_time',
    right_on='flint_time',
    direction='nearest',           # choose the closest future/previous measurement
    tolerance=pd.Timedelta('2H')   # max distance allowed (example: 2 hours)
)


/tmp/ipython-input-2652264407.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  tolerance=pd.Timedelta('2H')   # max distance allowed (example: 2 hours)


In [ ]:
df_merged = df_merged.dropna(subset=["target_tmpf", "tmpf"]) #drop any row without a temperature value
#drop target_time, flint_time as they are no longer needed, drop station as it is an id

df_final = df_merged.drop(columns=["station_y", "target_time", "flint_time", "lon_y", "lat_y",
"elevation_y","dwpf_y","relh_y","feel_y","drct_y","sped_y","mslp_y","DOY_y","TOD_y"]) #"station_x" used to be on here but removed


In [ ]:
print(df_final.columns.tolist())

# ---- 1. Floor timestamps to hourly buckets ----
df_final["hour"] = df_final["datetime"].dt.floor("H")

# ---- 2. Keep only needed columns ----
df_small = df_final[["station_x", "hour", "tmpf", "dwpf_x"]].copy()

# ---- 3. Pivot so each station becomes columns ----
df_wide = df_small.pivot_table(
    index="hour",
    columns="station_x",
    values=["tmpf", "dwpf_x"],
    aggfunc="mean"
)

# Flatten column names: ('tmpf','FNT') -> 'FNT_tmpf'
df_wide.columns = [f"{station}_{var}" for var, station in df_wide.columns]
df_wide = df_wide.reset_index()

['station_x', 'datetime', 'lon_x', 'lat_x', 'elevation_x', 'tmpf', 'dwpf_x', 'relh_x', 'feel_x', 'drct_x', 'sped_x', 'mslp_x', 'DOY_x', 'TOD_x', 'target_tmpf']


/tmp/ipython-input-514258834.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_final["hour"] = df_final["datetime"].dt.floor("H")


In [ ]:
# print(df_merged.columns.tolist())
# print(df_final.head(3))

print(df_wide.columns.tolist())
print(df_wide.shape)
print(df_wide.head())


['hour', '04W_dwpf_x', '14Y_dwpf_x', '2WX_dwpf_x', '7L2_dwpf_x', '8D3_dwpf_x', 'ABR_dwpf_x', 'AEL_dwpf_x', 'AIT_dwpf_x', 'ALO_dwpf_x', 'ANW_dwpf_x', 'ATY_dwpf_x', 'AXN_dwpf_x', 'AZO_dwpf_x', 'BAC_dwpf_x', 'BCK_dwpf_x', 'BFF_dwpf_x', 'BIS_dwpf_x', 'BKX_dwpf_x', 'BUU_dwpf_x', 'C29_dwpf_x', 'CID_dwpf_x', 'CKC_dwpf_x', 'CMX_dwpf_x', 'CNC_dwpf_x', 'CUT_dwpf_x', 'CVX_dwpf_x', 'D07_dwpf_x', 'DBQ_dwpf_x', 'DEH_dwpf_x', 'DNS_dwpf_x', 'DVN_dwpf_x', 'DXX_dwpf_x', 'EOK_dwpf_x', 'EST_dwpf_x', 'EZS_dwpf_x', 'FAR_dwpf_x', 'FFL_dwpf_x', 'FFM_dwpf_x', 'FLD_dwpf_x', 'FNT_dwpf_x', 'FSD_dwpf_x', 'FSE_dwpf_x', 'GRN_dwpf_x', 'GRR_dwpf_x', 'HEI_dwpf_x', 'HON_dwpf_x', 'IEN_dwpf_x', 'IKV_dwpf_x', 'ISQ_dwpf_x', 'IWD_dwpf_x', 'LBF_dwpf_x', 'LCG_dwpf_x', 'LNL_dwpf_x', 'LUM_dwpf_x', 'MBG_dwpf_x', 'MBL_dwpf_x', 'MCW_dwpf_x', 'MFI_dwpf_x', 'MKG_dwpf_x', 'MTW_dwpf_x', 'OEO_dwpf_x', 'OTG_dwpf_x', 'PBH_dwpf_x', 'PCZ_dwpf_x', 'PDC_dwpf_x', 'PNM_dwpf_x', 'RCA_dwpf_x', 'RDK_dwpf_x', 'RDR_dwpf_x', 'ROX_dwpf_x', 'RYV_dwpf_x

In [ ]:
forest = RandomForestRegressor(n_estimators=75, n_jobs=-1)


In [ ]:
#Manage the DOY
df_wide["DOY"] = df_wide["hour"].dt.dayofyear


In [ ]:
#do some offsetting  === lagging!

# 1. Sort by hour to ensure the shift happens chronologically
df_wide = df_wide.sort_values("hour")

# 2. Identify the columns we want to lag (all station temps and dewpoints)
# We exclude 'hour' and 'DOY'
feature_cols = [col for col in df_wide.columns if col not in ['hour', 'DOY', 'timestamp']]

# 3. Create lags (e.g., 1 hour ago and 3 hours ago)
for col in feature_cols:
    df_wide[f"{col}_lag1"] = df_wide[col].shift(1)
    df_wide[f"{col}_lag2"] = df_wide[col].shift(2)
    df_wide[f"{col}_lag3"] = df_wide[col].shift(3)
    df_wide[f"{col}_lag4"] = df_wide[col].shift(4)
    df_wide[f"{col}_lag6"] = df_wide[col].shift(6)

# 4. Drop the first few rows that now have NaN values due to the shift
df_wide = df_wide.dropna()

/tmp/ipython-input-2449509476.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_wide[f"{col}_lag3"] = df_wide[col].shift(3)
/tmp/ipython-input-2449509476.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_wide[f"{col}_lag4"] = df_wide[col].shift(4)
/tmp/ipython-input-2449509476.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

In [ ]:
# create DF final from flint

# Extract Flint hourly temps
flint = (
    df[df["station"] == "FNT"]
    .copy()
    .assign(hour=lambda x: x["datetime"].dt.floor("H"))
    .groupby("hour")["tmpf"]
    .mean()
    .rename("target_tmpf")
    .reset_index()
)

# Shift target 24 hours into the future
flint["hour"] = flint["hour"] - pd.Timedelta(hours=24)

# Merge target onto predictors
df_final = pd.merge(
    df_wide,
    flint,
    on="hour",
    how="inner"
)
df_final["year"] = df_final["hour"].dt.year

print(df_final.shape)
print(df_final.head())

# df_final["year"] = df_final["datetime"].dt.year
# df_final = df_final.drop(columns=["datetime"])
# df_final = df_final.drop(columns=["station_x"])

# Test set = only 2018 and 2021
df_test = df_final[df_final["year"].isin([2018, 2021])]
X_test = df_test.drop(columns=["target_tmpf", "year", "hour"])
y_test = df_test["target_tmpf"]

# Train set = every other year
df_train = df_final[~df_final["year"].isin([2018, 2021])]
X_train = df_train.drop(columns=["target_tmpf", "year", "hour"])
y_train = df_train["target_tmpf"]



(3407, 976)
                 hour  04W_dwpf_x  14Y_dwpf_x  2WX_dwpf_x  7L2_dwpf_x  \
0 2017-05-09 18:00:00        36.8        36.2        47.0        38.6   
1 2017-05-10 01:00:00        41.0        42.2        46.0        50.6   
2 2017-05-10 03:00:00        41.0        42.8        43.0        44.6   
3 2017-05-10 04:00:00        41.0        42.8        43.0        45.8   
4 2017-05-10 05:00:00        41.0        42.2        42.0        44.6   

   8D3_dwpf_x  ABR_dwpf_x  AEL_dwpf_x  AIT_dwpf_x  ALO_dwpf_x  ...  \
0        34.0        32.0        44.6        27.8        54.0  ...   
1        39.0        42.0        47.0        37.4        53.0  ...   
2        44.0        43.0        44.0        35.6        53.0  ...   
3        43.0        44.0        45.2        35.6        53.0  ...   
4        44.0        44.0        45.8        35.6        53.0  ...   

   Y51_tmpf_lag3  Y51_tmpf_lag4  Y51_tmpf_lag6  YKN_tmpf_lag1  YKN_tmpf_lag2  \
0      50.033333      48.300000      47.833333  

/tmp/ipython-input-243573227.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .assign(hour=lambda x: x["datetime"].dt.floor("H"))


In [ ]:
# forest.fit(df_train.drop(columns=["target_tmpf"]), df_train["target_tmpf"])
# y_pred = forest.predict(df_test.drop(columns=["target_tmpf"]))

forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)



In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
# accuracy = accuracy_score(y_test, y_pred)

print("RMSE:", rmse)
print("R²:", r2)


RMSE: 5.746126677055911
R²: 0.9316247859507893


In [ ]:
absolute_error = np.abs(y_test - y_pred)
within_3_degrees = np.sum(absolute_error <= 5)
percentage_within_3 = (within_3_degrees / len(y_test)) * 100
print(f"Percentage of forecasts accurate within 5 degrees: {percentage_within_3:.2f}%")

within_3_degrees = np.sum(absolute_error <= 3)
percentage_within_3 = (within_3_degrees / len(y_test)) * 100
print(f"Percentage of forecasts accurate within 3 degrees: {percentage_within_3:.2f}%")

within_3_degrees = np.sum(absolute_error <= 2)
percentage_within_3 = (within_3_degrees / len(y_test)) * 100
print(f"Percentage of forecasts accurate within 2 degrees: {percentage_within_3:.2f}%")

within_3_degrees = np.sum(absolute_error <= 1)
percentage_within_3 = (within_3_degrees / len(y_test)) * 100
print(f"Percentage of forecasts accurate within 1 degrees: {percentage_within_3:.2f}%")


Percentage of forecasts accurate within 5 degrees: 68.31%
Percentage of forecasts accurate within 3 degrees: 50.05%
Percentage of forecasts accurate within 2 degrees: 33.44%
Percentage of forecasts accurate within 1 degrees: 17.33%


In [ ]:
#get feature importances
importances = forest.feature_importances_
for name, val in sorted(zip(X_train.columns, importances), key=lambda x: -x[1]):
    print(name, val)


CKC_tmpf 0.22723903669990728
MFI_tmpf 0.21843258061502163
SAW_tmpf 0.17799444230144187
AXN_tmpf 0.03132003814123397
EST_tmpf 0.027258251518409708
GRR_tmpf 0.02246983836027413
DEH_tmpf 0.021006407606111818
14Y_tmpf 0.018331209573086628
SYN_tmpf 0.015281344821425276
AEL_tmpf 0.014787232306972847
MCW_dwpf_x 0.011508216738803706
AIT_tmpf 0.009808055869505918
CKC_tmpf_lag1 0.009576212927853989
OEO_tmpf 0.00929367606976159
DVN_tmpf 0.008272589719635253
AZO_tmpf 0.007324872907731146
DOY 0.005190178661695565
LNL_tmpf 0.00459979857595721
Y51_tmpf 0.004591611144622929
FSE_tmpf 0.0042050961102387385
SUE_tmpf 0.004169455241910429
IWD_tmpf 0.004157450590886714
CID_tmpf 0.003990596679878826
MKG_tmpf 0.00384615095284959
PBH_dwpf_x 0.003763784679984708
PBH_tmpf 0.003096595032324888
PNM_tmpf 0.0027407912601609893
XVG_tmpf 0.0022942338521520665
EZS_tmpf 0.002010561719302712
MTW_tmpf 0.0020012871195140224
FAR_tmpf 0.001849553508039936
ROX_tmpf 0.0018412702342986715
ALO_tmpf 0.0018124961769363057
AEL_dwpf